In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
import argparse
import h5py
import numpy as np
import pandas as pd
from sklearn import preprocessing
import os
import time
from pathlib import Path
from urllib.request import urlretrieve
import logging
from li.Baseline import Baseline
from li.LearnedIndex import LearnedIndex
from li.utils import save_as_pickle
from li.model import data_X_to_torch


In [3]:
np.random.seed(2023)

logging.basicConfig(
    level=logging.INFO,
    format='[%(asctime)s][%(levelname)-5.5s][%(name)-.20s] %(message)s'
)
LOG = logging.getLogger(__name__)


In [4]:
kind='pca32v2'
size='100K'
key='pca32'

In [5]:
data = np.array(h5py.File(os.path.join("data", kind, size, "dataset.h5"), "r")[key])
queries = np.array(h5py.File(os.path.join("data", kind, size, "query.h5"), "r")[key])

In [6]:
kind_search = 'clip768v2'
key_search = 'emb'
data_search = np.array(
    h5py.File(os.path.join("data", kind_search, size, "dataset.h5"), "r")[key_search]
)

In [7]:
queries_search = np.array(
    h5py.File(os.path.join("data", kind_search, size, "query.h5"), "r")[key_search]
)

In [8]:
data = pd.DataFrame(data)
data.index += 1

data_search = pd.DataFrame(data_search)
data_search.index += 1

In [9]:
import torch
from torch import nn
import torch.nn.functional as nnf
import numpy as np
from li.Logger import Logger
from typing import Tuple
import torch.utils.data

torch.manual_seed(2023)
np.random.seed(2023)

class Model(nn.Module):
    def __init__(self, input_dim=768, output_dim=1000, model_type=None):
        super().__init__()
        if model_type == 'MLP':
            self.layers = torch.nn.Sequential(
                torch.nn.Linear(input_dim, 128),
                torch.nn.ReLU(),
                torch.nn.Linear(128, output_dim)
            )
        if model_type == 'MLP-2':
            self.layers = torch.nn.Sequential(
                torch.nn.Linear(input_dim, 64),
                torch.nn.ReLU(),
                torch.nn.Linear(64, output_dim)
            )
        if model_type == 'MLP-3':
            self.layers = torch.nn.Sequential(
                torch.nn.Linear(input_dim, 256),
                torch.nn.ReLU(),
                torch.nn.Linear(256, output_dim)
            )
        if model_type == 'MLP-4':
            self.layers = torch.nn.Sequential(
                torch.nn.Linear(input_dim, 512),
                torch.nn.ReLU(),
                torch.nn.Linear(512, output_dim)
            )
        if model_type == 'MLP-5':
            self.layers = torch.nn.Sequential(
                torch.nn.Linear(input_dim, 256),
                torch.nn.ReLU(),
                torch.nn.Linear(256, 128),
                torch.nn.ReLU(),
                torch.nn.Linear(128, output_dim)
            )
        if model_type == 'MLP-6':
            self.layers = torch.nn.Sequential(
                torch.nn.Linear(input_dim, 32),
                torch.nn.ReLU(),
                torch.nn.Linear(32, output_dim)
            )
        if model_type == 'MLP-7':
            self.layers = torch.nn.Sequential(
                torch.nn.Linear(input_dim, 16),
                torch.nn.ReLU(),
                torch.nn.Linear(16, output_dim)
            )
        if model_type == 'MLP-8':
            self.layers = torch.nn.Sequential(
                torch.nn.Linear(input_dim, 8),
                torch.nn.ReLU(),
                torch.nn.Linear(8, output_dim)
            )
        if model_type == 'MLP-9':
            self.layers = torch.nn.Sequential(
                torch.nn.Linear(input_dim, 8),
                torch.nn.ReLU(),
                torch.nn.Linear(input_dim, 16),
                torch.nn.ReLU(),
                torch.nn.Linear(16, output_dim)
            )
        self.n_output_neurons = output_dim

    def forward(self, x: torch.FloatTensor) -> torch.FloatTensor:
        outputs = self.layers(x)
        return outputs


def data_X_to_torch(data) -> torch.FloatTensor:
    """ Creates torch training data."""
    data_X = torch.from_numpy(np.array(data).astype(np.float32))
    return data_X


def data_to_torch(data, labels) -> Tuple[torch.FloatTensor, torch.LongTensor]:
    """ Creates torch training data and labels."""
    data_X = data_X_to_torch(data)
    data_y = torch.as_tensor(torch.from_numpy(labels), dtype=torch.long)
    return data_X, data_y


def get_device() -> torch.device:
    """ Gets the `device` to be used by torch.
    This arugment is needed to operate with the PyTorch model instance.

    Returns
    ------
    torch.device
        Device
    """
    use_cuda = torch.cuda.is_available()
    device = torch.device('cuda:0' if use_cuda else 'cpu')
    torch.backends.cudnn.benchmark = True
    return device


class NeuralNetwork(Logger):
    """ The neural network class corresponding to every inner node.

    Parameters
    ----------
    input_dim : int
        The input dimension.
    output_dim : int
        The output dimension.
    loss : torch.nn, optional
        The loss function, the default is torch.nn.CrossEntropyLoss.
    lr : float, optional
        The learning rate, the default is 0.001.
    model_type : str, optional
        The model type, the default is 'MLP'.
    class_weight : torch.FloatTensor, optional
        The class weights, the default is None.
    """
    def __init__(
        self,
        input_dim,
        output_dim,
        loss=torch.nn.CrossEntropyLoss,
        lr=0.1,
        model_type='MLP',
        class_weight=None
    ):
        self.device = get_device()
        self.model = Model(input_dim, output_dim, model_type=model_type).to(self.device)
        if not isinstance(class_weight, type(None)):
            self.loss = loss(weight=class_weight.to(self.device))
        else:
            self.loss = loss()
        self.optimizer = torch.optim.Adam(self.model.parameters(), lr=lr)

    def train(
        self,
        data_X: torch.FloatTensor,
        data_y: torch.LongTensor,
        epochs=500,
        logger=None
    ):
        step = epochs // 10
        losses = []
        if logger:
            logger.info(f'Epochs: {epochs}, step: {step}')
        for ep in range(epochs):
            pred_y = self.model(data_X.to(self.device))
            curr_loss = self.loss(pred_y, data_y.to(self.device))
            if ep % step == 0 and ep != 0:
                if logger:
                    logger.info(f'Epoch {ep} | Loss {curr_loss.item()}')
            losses.append(curr_loss.item())

            self.model.zero_grad()
            curr_loss.backward()

            self.optimizer.step()
        return losses

    def train_batch(
        self,
        dataset,
        epochs=5,
        logger=None
    ):
        step = epochs // 10
        step = step if step > 0 else 1
        losses = []
        if logger:
            logger.info(f'Epochs: {epochs}, step: {step}')
        for ep in range(epochs):
            for data_X, data_y in iter(dataset):
                pred_y = self.model(data_X.to(self.device))
                curr_loss = self.loss(pred_y, data_y.to(self.device))

            if ep % step == 0 and ep != 0:
                if logger:
                    logger.info(f'Epoch {ep} | Loss {curr_loss.item():.5f}')
            losses.append(curr_loss.item())

            self.model.zero_grad()
            curr_loss.backward()

            self.optimizer.step()
        return losses

    def predict(self, data_X: torch.FloatTensor):
        """ Collects predictions for multiple data points (used in structure building)."""
        self.model = self.model.to(self.device)
        self.model.eval()

        all_outputs = torch.tensor([], device=self.device)
        with torch.no_grad():
            outputs = self.model(data_X.to(self.device))
            all_outputs = torch.cat((all_outputs, outputs), 0)

        _, y_pred = torch.max(all_outputs, 1)
        return y_pred.cpu().numpy()

    def predict_proba(self, data_X: torch.FloatTensor):
        """ Collects predictions for a single data point (used in query predictions)."""
        self.model = self.model.to(self.device)
        self.model.eval()

        with torch.no_grad():
            outputs = self.model(data_X.to(self.device))

        if outputs.dim() == 1:
            dim = 0
        else:
            dim = 1
        prob = nnf.softmax(outputs, dim=dim)
        probs, classes = prob.topk(prob.shape[1])

        return probs.cpu().numpy(), classes.cpu().numpy()


class LIDataset(torch.utils.data.Dataset):
    def __init__(self, dataset_x, dataset_y):
        self.dataset_x, self.dataset_y = data_to_torch(dataset_x, dataset_y)

    def __len__(self):
        return self.dataset_x.shape[0]

    def __getitem__(self, idx):
        return self.dataset_x[idx-1], self.dataset_y[idx-1]


In [10]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import time

def pairwise_cosine_threshold(x, y, threshold, cat_idxs, k=10):
    s = time.time()
    result = 1-cosine_similarity(x, y)
    t_pure_seq_search = time.time() - s
    # create an array of consisten shapes
    #print(result.shape)
    #print(threshold.shape, threshold[cat_idxs].shape)
    thresh_consistent = np.repeat(threshold[cat_idxs, np.newaxis], result.shape[1], 1)
    relevant_dists = np.where(result < thresh_consistent)
    # filter the relevant object ids
    try:
        relevant_object_ids = np.unique(relevant_dists[1])
        max_idx = relevant_object_ids.shape[0]
        if max_idx == 0:
            return None, t_pure_seq_search
    except ValueError:
        # There is no distance below the threshold, we can return
        return None, t_pure_seq_search
    max_idx = max_idx if max_idx > k else k
    # output array filled with some large value
    output_arr = np.full(shape=(result.shape[0], max_idx), fill_value=10_000, dtype=np.float)
    #index_df = pd.DataFrame(relevant_dists[0])
    #index_df = pd.DataFrame(relevant_dists[0], relevant_dists[1]).reset_index()
    # create indexes to store the relevant distances
    #index_df['mapping'] = index_df.groupby('index').ngroup()
    mapping = dict(zip(relevant_object_ids, np.arange(relevant_object_ids.shape[0])))
    # tried also with np.vectorize, wasn't faster
    output_arr_2nd_dim = np.array([mapping[x] for x in relevant_dists[1]])
    to_be_added = result[relevant_dists[0], relevant_dists[1]]
    # populate the output array
    output_arr[relevant_dists[0], output_arr_2nd_dim] = to_be_added
    return output_arr, relevant_object_ids, t_pure_seq_search

In [11]:
import numpy as np
from li.Logger import Logger
from li.utils import pairwise_cosine
import time
import torch
import torch.utils.data
import faiss
from tqdm import tqdm
import numpy as np
torch.manual_seed(2023)
np.random.seed(2023)

class LearnedIndex(Logger):

    def __init__(self):
        self.pq = []
        self.model = None

    def search(
        self,
        data_navigation,
        queries_navigation,
        data_search,
        queries_search,
        pred_categories,
        n_buckets=1,
        k=10,
        use_threshold=False
    ):
        """ Search for k nearest neighbors for each query in queries.

        Parameters
        ----------
        queries : np.array
            Queries to search for.
        data : np.array
            Data to search in.
        n_buckets : int
            Number of most similar buckets to search in.
        k : int
            Number of nearest neighbors to search for.

        Returns
        -------
        dists : np.array
            Array of shape (queries.shape[0], k) with distances to nearest neighbors for each query.
        anns : np.array
            Array of shape (queries.shape[0], k) with nearest neighbors for each query.
        time : float
            Time it took to search.
        """
        assert self.model is not None, 'Model is not trained, call `build` first.'
        s = time.time()
        _, pred_proba_categories = self.model.predict_proba(
            data_X_to_torch(queries_navigation)
        )
        t_inference = time.time() - s
        anns_final = None
        dists_final = None
        # sorts the predictions of a bucket for each query, ordered by lowest probability
        data_navigation['category'] = pred_categories

        # iterates over the predicted buckets starting from the most similar (index -1)
        t_all_buckets = 0
        t_all_pairwise = 0
        t_all_sort = 0
        t_all_pure_pairwise = 0
        t_comp_threshold = 0
        for bucket in range(n_buckets):
            if bucket != 0 and use_threshold:
                s_ = time.time()
                threshold_dist = dists_final.max(axis=1)
                t_comp_threshold += time.time() - s_
            else:
                threshold_dist = None
            dists, anns, t_all, t_pairwise, t_pure_pairwise, t_sort = self.search_single(
                data_navigation,
                data_search,
                queries_search,
                pred_proba_categories[:, bucket],
                threshold_dist=threshold_dist
            )
            print(bucket, anns[2], dists[2])
            t_all_buckets += t_all
            t_all_pairwise += t_pairwise
            t_all_pure_pairwise += t_pure_pairwise
            t_all_sort += t_sort
            if anns_final is None:
                anns_final = anns
                dists_final = dists
            else:
                # stacks the results from the previous sorted anns and dists
                # *_final arrays now have shape (queries.shape[0], k*2)
                anns_final = np.hstack((anns_final, anns))
                dists_final = np.hstack((dists_final, dists))
                # gets the sorted indices of the stacked dists
                idx_sorted = dists_final.argsort(kind='stable', axis=1)[:, :k]
                # indexes the final arrays with the sorted indices
                # *_final arrays now have shape (queries.shape[0], k)
                idx = np.ogrid[tuple(map(slice, dists_final.shape))]
                idx[1] = idx_sorted
                dists_final = dists_final[tuple(idx)]
                anns_final = anns_final[tuple(idx)]

                assert anns_final.shape == dists_final.shape == (queries_search.shape[0], k)

        self.logger.info(f't_comp_threshold: {t_comp_threshold}')
        return dists_final, anns_final, time.time() - s, t_inference, t_all_buckets, t_all_pairwise, t_all_pure_pairwise, t_all_sort

    def search_single(
        self,
        data_navigation,
        data_search,
        queries_search,
        pred_categories,
        k=10,
        threshold_dist=None
    ):
        """ Search for k nearest neighbors for each query in queries.

        Parameters
        ----------
        queries : np.array
            Queries to search for.
        data : np.array
            Data to search in.
        k : int
            Number of nearest neighbors to search for.

        Returns
        -------
        anns : np.array
            Array of shape (queries.shape[0], k) with nearest neighbors for each query.
        final_dists_k : np.array
            Array of shape (queries.shape[0], k) with distances to nearest neighbors for each query.
        time : float
            Time it took to search.
        """
        s_all = time.time()
        nns = np.zeros((queries_search.shape[0], k), dtype=np.uint32)
        dists = np.full((queries_search.shape[0], k), fill_value=10_000, dtype=np.float32)

        if 'category' in data_search.columns:
            data_search = data_search.drop('category', axis=1, errors='ignore')

        t_pairwise = 0
        t_pure_pairwise = 0
        t_sort = 0
        for cat, g in tqdm(data_navigation.groupby('category')):
            cat_idxs = np.where(pred_categories == cat)[0]
            bucket_obj_indexes = g.index
            if bucket_obj_indexes.shape[0] != 0 and cat_idxs.shape[0] != 0:
                s = time.time()
                # TODO: Add filter, filter will be different for every query
                # OR pass nns, dists from previous buckets
                if threshold_dist is not None:
                    seq_search_dists = pairwise_cosine_threshold(
                        queries_search[cat_idxs],
                        data_search.loc[bucket_obj_indexes],
                        threshold_dist,
                        cat_idxs,
                        k
                    )
                    if seq_search_dists[0] is None:
                        t_pure_pairwise += seq_search_dists[1]
                        # There is no distance below the threshold, we can continue
                        continue
                    else:
                        # seq_search_dists[1] contains the indexes of the relevant objects
                        bucket_obj_indexes = bucket_obj_indexes[seq_search_dists[1]]
                        t_pure_pairwise += seq_search_dists[2]
                        seq_search_dists = seq_search_dists[0]
                else:
                    s_ = time.time()
                    seq_search_dists = pairwise_cosine(
                        queries_search[cat_idxs],
                        data_search.loc[bucket_obj_indexes]
                    )
                    t_pure_pairwise += time.time() - s_
                t_pairwise += time.time() - s
                s = time.time()
                ann_relative = seq_search_dists.argsort(kind='quicksort')[
                    :, :k if k < seq_search_dists.shape[1] else seq_search_dists.shape[1]
                ]
                t_sort += time.time() - s
                if bucket_obj_indexes.shape[0] < k:
                    # pad to `k` if needed
                    pad_needed = (k - bucket_obj_indexes.shape[0]) // 2 + 1
                    bucket_obj_indexes = np.pad(np.array(bucket_obj_indexes), pad_needed, 'edge')[:k]
                    ann_relative = np.pad(ann_relative[0], pad_needed, 'edge')[:k].reshape(1, -1)
                    seq_search_dists = np.pad(seq_search_dists[0], pad_needed, 'edge')[:k].reshape(1, -1)
                    _, i = np.unique(seq_search_dists, return_index=True)
                    duplicates_i = np.setdiff1d(np.arange(k), i)
                    # assign a large number such that the duplicated value gets replaced
                    seq_search_dists[0][duplicates_i] = 10_000

                nns[cat_idxs] = np.array(bucket_obj_indexes)[ann_relative]
                dists[cat_idxs] = np.take_along_axis(seq_search_dists, ann_relative, axis=1)
        t_all = time.time() - s_all
        return dists, nns, t_all, t_pairwise, t_pure_pairwise, t_sort

    
    def build(self, data, n_categories=100, epochs=100, lr=0.1, model_type='MLP', n_levels=1):
        """ Build the index.

        Parameters
        ----------
        data : np.array
            Data to build the index on.

        Returns
        -------
        time : float
            Time it took to build the index.
        """
        s = time.time()
        # ---- cluster the data into categories ---- #
        _, labels = self.cluster(data, n_categories)

        # ---- train a neural network ---- #
        dataset = LIDataset(data, labels)
        train_loader = torch.utils.data.DataLoader(
            dataset,
            batch_size=256,
            sampler=torch.utils.data.SubsetRandomSampler(
                data.index.values.tolist()
            )
        )
        nn = NeuralNetwork(
            input_dim=data.shape[1],
            output_dim=n_categories,
            lr=lr,
            model_type=model_type
        )
        nn.train_batch(train_loader, epochs=epochs, logger=self.logger)
        self.model = nn
        # ---- collect predictions ---- #
        if n_levels == 1:
            return nn.predict(data_X_to_torch(data)), time.time() - s
        else:
            predict_categories = nn.predict(data_X_to_torch(data))
            # ---- train a neural network for each category ---- #
            models = []
            predict_categories_l2 = pd.DataFrame([], index=data.index)
            predict_categories_l2['L2'] = None
            for cat in range(n_categories):
                cat_data = data.loc[predict_categories == cat]
                #cat_labels = predict_categories[labels == cat]
                _, cat_labels = self.cluster(cat_data, n_categories)
                dataset = LIDataset(cat_data, cat_labels)
                train_loader = torch.utils.data.DataLoader(
                    dataset,
                    batch_size=256,
                    sampler=torch.utils.data.SubsetRandomSampler(
                        np.arange(cat_data.index.shape[0]).tolist()# + 1
                        #c#at_data.index.values.tolist()
                    )
                )
                nn = NeuralNetwork(
                    input_dim=cat_data.shape[1],
                    output_dim=n_categories,
                    lr=lr,
                    model_type=model_type
                )
                nn.train_batch(train_loader, epochs=epochs, logger=self.logger)
                models.append(nn)
                predict_categories_l2.loc[cat_data.index, 'L2'] = nn.predict(data_X_to_torch(cat_data))
            self.models = models
            return self.model.predict(data_X_to_torch(data)), predict_categories_l2, time.time() - s


    def cluster(
        self,
        data,
        n_clusters,
        n_redo=10,
        spherical=True,
        int_centroids=True,

    ):
        if data.shape[0] < 2:
            return None, np.zeros_like(data.shape[0])

        if data.shape[0] < n_clusters:
            n_clusters = data.shape[0] // 5
            if n_clusters < 2:
                n_clusters = 2

        kmeans = faiss.Kmeans(
            d=np.array(data).shape[1],
            k=n_clusters,
            verbose=True,
            #nredo=n_redo,
            #spherical=spherical,
            #int_centroids=int_centroids,
            #update_index=False,
            seed=2023
        )
        X = np.array(data).astype(np.float32)
        kmeans.train(X)

        return kmeans, kmeans.index.search(X, 1)[1].T[0]


[2023-07-14 16:43:40,410][INFO ][faiss.loader] Loading faiss with AVX2 support.
[2023-07-14 16:43:40,413][INFO ][faiss.loader] Could not load library with AVX2 support due to:
ModuleNotFoundError("No module named 'faiss.swigfaiss_avx2'",)
[2023-07-14 16:43:40,415][INFO ][faiss.loader] Loading faiss.
[2023-07-14 16:43:40,462][INFO ][faiss.loader] Successfully loaded faiss.


In [12]:
li = LearnedIndex()
pred_categories, build_t = li.build(
    data,
    n_categories=120,
    epochs=5,
    lr=0.1,
    n_levels=1
)

[2023-07-14 16:43:41,243][INFO ][__main__.LearnedInde] Epochs: 5, step: 1
[2023-07-14 16:43:45,582][INFO ][__main__.LearnedInde] Epoch 1 | Loss 4.72351
[2023-07-14 16:43:47,782][INFO ][__main__.LearnedInde] Epoch 2 | Loss 4.69153
[2023-07-14 16:43:49,900][INFO ][__main__.LearnedInde] Epoch 3 | Loss 4.32741
[2023-07-14 16:43:51,897][INFO ][__main__.LearnedInde] Epoch 4 | Loss 4.13195


In [13]:
queries_navigation = queries
data_navigation = data

In [14]:
data_navigation['category'] = pred_categories

In [15]:
_, pred_proba_categories = li.model.predict_proba(
    data_X_to_torch(queries_navigation)
)

In [16]:
k = 10

In [17]:
bucket = 6

In [18]:
loaded_gt[2][:10]

NameError: name 'loaded_gt' is not defined

In [19]:
dists, nns, search_t, inference_t, search_single_t, seq_search_t, pure_seq_search_t, sort_t = li.search(
    data_navigation=data,
    queries_navigation=queries,
    data_search=data_search,
    queries_search=queries_search,
    pred_categories=pred_categories,
    n_buckets=bucket,
    k=k,
    use_threshold=True
)

  4%|▍         | 2/46 [00:00<00:03, 13.72it/s]

0 [51954 87690  3444  5491 65271 92521 58848 72008 63515 68040] [0.13460062 0.14457192 0.15563862 0.15744463 0.15815064 0.16120386
 0.1637611  0.16689926 0.16764964 0.17210594]


  4%|▍         | 2/46 [00:00<00:03, 12.80it/s]

1 [  398 64782 64673 64598 64406 64140 64130 63814 63241 63111] [10000. 10000. 10000. 10000. 10000. 10000. 10000. 10000. 10000. 10000.]


  4%|▍         | 2/46 [00:00<00:03, 13.78it/s]

...

3 [  409 65511 65529 65656 65818 66190 66192 66225 65505 66464] [10000. 10000. 10000. 10000. 10000. 10000. 10000. 10000. 10000. 10000.]


  4%|▍         | 2/46 [00:00<00:03, 12.15it/s]

4 [0 0 0 0 0 0 0 0 0 0] [10000. 10000. 10000. 10000. 10000. 10000. 10000. 10000. 10000. 10000.]


100%|██████████| 46/46 [00:05<00:00,  8.18it/s]
[2023-07-14 16:45:07,076][INFO ][__main__.LearnedInde] t_comp_threshold: 0.0023717880249023438


5 [0 0 0 0 0 0 0 0 0 0] [10000. 10000. 10000. 10000. 10000. 10000. 10000. 10000. 10000. 10000.]


In [23]:
dists

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [32]:
loaded_gt[0][:10]

array([79172, 15735, 22337,   231, 74173, 41079, 38159, 71849, 69015,
       92811], dtype=int32)

In [23]:
LOG.info(f'Loading GT')
gt_path = f'data/groundtruth-{size}.h5'
f3 = h5py.File(gt_path, 'r')
loaded_gt = f3['knns'][:, :]

[2023-07-14 16:45:15,450][INFO ][__main__] Loading GT


In [24]:
def get_recall(I, gt, k):
    assert k <= I.shape[1]
    assert len(I) == len(gt)

    n = len(I)
    recall = 0
    for i in range(n):
        recall += len(set(I[i, :k]) & set(gt[i, :k]))
    return recall / (n * k)

recall = get_recall(nns, loaded_gt, 10)

In [25]:
recall

0.89478

In [ ]:
[[5.8051813e-02 3.6613084e-02 3.6324631e-02 ... 4.9363379e-04
  4.1887566e-04 3.1460408e-04]
 [6.7765884e-02 4.8477888e-02 4.6616543e-02 ... 2.8254880e-04
  2.2255247e-04 1.7899058e-04]
 [2.6327980e-01 1.1470813e-01 5.6388140e-02 ... 1.2473087e-04
  6.4560751e-05 5.8199759e-05]
 ...
 [9.9357553e-02 4.0632725e-02 3.5712827e-02 ... 3.6862367e-04
  3.5981109e-04 2.7229940e-04]
 [6.6355608e-02 3.7934203e-02 3.7579723e-02 ... 3.9285724e-04
  3.0156417e-04 2.0785566e-04]
 [3.3466626e-02 3.2320090e-02 2.8980738e-02 ... 1.2179379e-03
  1.1678757e-03 7.6151139e-04]]

In [29]:
for n in range(10_000):
    if not np.all(nns[n] == loaded_gt[n][:10]):
        print(n)
        break

2


In [20]:
nns[2]

array([51954, 87690,  3444,  5491, 65271, 92521, 58848, 72008, 63515,
       68040], dtype=uint32)

In [31]:
loaded_gt[2][:10]

array([51954, 87690,  3444,  5491, 65271, 92521, 58848, 72008, 63515,
       68040], dtype=int32)

In [33]:
data_navigation.loc[79172][['category', 'category_L2']]

category       6
category_L2    4
Name: 79172, dtype: object

In [36]:
queries[0]

array([-0.06877425, -0.11037602,  0.05646035, -0.11508854, -0.03577028,
        0.09110859,  0.07477964,  0.10778218,  0.04116842,  0.00877987,
       -0.06257925, -0.17031246,  0.00780125, -0.099485  ,  0.04967897,
       -0.12064676,  0.0035742 , -0.07642894,  0.08400699, -0.03188556,
       -0.01020392,  0.07147753, -0.01378688, -0.10539152, -0.00040444,
       -0.00291365,  0.12945361,  0.05624911,  0.00785595,  0.01020052,
        0.06252866, -0.00992952], dtype=float32)

In [22]:
cat = ((0, 2))
cat_idxs = np.where(pred_proba_categories[:, 0] == cat[0])[0]
cat_idxs

array([   2,   11,   14,   21,   29,   64,   66,   67,   81,   89,   96,
        100,  117,  137,  148,  179,  181,  188,  203,  217,  221,  222,
        233,  249,  255,  279,  284,  289,  290,  296,  297,  337,  347,
        357,  377,  402,  404,  419,  421,  431,  434,  437,  442,  455,
        459,  470,  481,  490,  497,  526,  532,  534,  552,  557,  584,
        589,  590,  623,  639,  642,  679,  690,  695,  709,  734,  739,
        758,  769,  778,  788,  793,  803,  809,  814,  835,  898,  908,
        922,  927,  930,  954,  958,  970,  972,  974,  978,  990,  997,
       1053, 1058, 1077, 1086, 1093, 1105, 1112, 1116, 1123, 1140, 1152,
       1156, 1167, 1174, 1188, 1190, 1245, 1251, 1262, 1268, 1271, 1273,
       1288, 1302, 1325, 1331, 1333, 1352, 1380, 1395, 1402, 1439, 1446,
       1451, 1478, 1488, 1498, 1530, 1551, 1559, 1566, 1592, 1599, 1610,
       1626, 1627, 1645, 1651, 1657, 1667, 1673, 1725, 1726, 1729, 1747,
       1764, 1769, 1799, 1800, 1809, 1835, 1841, 18

In [23]:
_, pred_proba_categories_l2 = li.models[cat[0]].predict_proba(
    data_X_to_torch(queries_navigation[cat_idxs])
)

In [24]:
cat_idxs2 = np.where(pred_proba_categories_l2[:, 0] == cat[1])[0]

In [26]:
queries_navigation[cat_idxs][cat_idxs2]

array([[-0.10336345, -0.01680509, -0.34836447, ...,  0.02956844,
        -0.005949  , -0.01537791],
       [-0.21702042,  0.04931362, -0.33564436, ...,  0.0011189 ,
        -0.00168799, -0.07086135],
       [-0.11757332,  0.00409064, -0.36369807, ...,  0.01488095,
         0.09611854, -0.00749078],
       ...,
       [-0.04934649, -0.13572182, -0.23093528, ..., -0.03342572,
         0.03512686, -0.02864448],
       [ 0.09900215, -0.04749763, -0.28956988, ..., -0.01773802,
        -0.02736606,  0.00395937],
       [-0.326379  ,  0.11535953, -0.37847796, ..., -0.00936712,
        -0.01018373,  0.03260171]], dtype=float32)

In [25]:
cat_idxs2

array([  0,   3,   6,   7,   9,  24,  26,  27,  32,  34,  37,  41,  46,
        52,  59,  62,  65,  66,  77,  82,  85,  86,  91,  92,  94,  99,
       103, 112, 116, 117, 128, 135, 136, 138, 140, 144, 145, 146, 152,
       155, 158, 159, 162, 165, 174, 178, 187, 188, 191, 192, 193, 194,
       195, 198, 199, 200, 203, 208, 209, 213, 220, 221, 224, 228, 229,
       232, 244, 245, 250, 253, 254, 257, 258, 262, 263, 264, 268, 270,
       275, 276, 277, 278, 286, 290, 293, 296, 298, 299, 302, 308, 313,
       315, 317, 323, 325, 326, 329, 330, 333, 334, 338, 344, 347, 349,
       351, 355, 359, 362, 367, 369, 379, 381, 389, 394, 397, 402, 405,
       408, 411, 421, 427, 434, 439, 440, 442, 445, 446, 447, 449, 450,
       453, 458, 465, 468, 471, 474, 477, 478, 481, 483, 486, 488, 490,
       493, 496, 499, 500, 503, 505, 506, 507, 509, 510, 511, 514, 519,
       522, 524, 530, 531, 532, 533, 536, 538, 540, 542, 543, 545, 548,
       550, 553, 554, 555, 556, 557, 561, 563, 576, 579, 580, 58

In [40]:
k = 10
nns = np.zeros((queries_search.shape[0], k), dtype=np.uint32)
dists = np.zeros((queries_search.shape[0], k), dtype=np.float32)

for cat, g in tqdm(data_navigation.groupby(['category', 'category_L2'])):
    if cat == ((6, 4)):
        cat_idxs = np.where(pred_proba_categories[:, 0] == cat[0])[0]
        print(cat_idxs.shape)
        _, pred_proba_categories_l2 = li.models[cat[0]].predict_proba(
            data_X_to_torch(queries_navigation[cat_idxs])
        )
        cat_idxs = np.where(pred_proba_categories_l2[:, 0] == cat[1])[0]
        bucket_obj_indexes = g.index
        print(cat_idxs.shape)
        #bucket_obj_indexes = g.index
        print(cat, g.index.shape)
        print(queries[cat_idxs])
        seq_search_dists = pairwise_cosine(
            queries_search[cat_idxs],
            data_search.loc[bucket_obj_indexes]
        )
        ann_relative = seq_search_dists.argsort(kind='quicksort')[
            :, :k if k < seq_search_dists.shape[1] else seq_search_dists.shape[1]
        ]
        nns[cat_idxs] = np.array(bucket_obj_indexes)[ann_relative]
        dists[cat_idxs] = np.take_along_axis(seq_search_dists, ann_relative, axis=1)

100%|██████████| 87/87 [00:00<00:00, 888.25it/s]

(1310,)
(197,)
(6, 4) (2188,)
[[-0.06877425 -0.11037602  0.05646035 ...  0.01020052  0.06252866
  -0.00992952]
 [-0.06634384 -0.00121936  0.14378567 ...  0.01600719 -0.05402476
  -0.00075561]
 [ 0.01517327  0.24030505 -0.12153216 ... -0.03119725 -0.00319142
   0.01317247]
 ...
 [ 0.1413608  -0.23707068  0.1034386  ... -0.06873898 -0.02459563
  -0.01605512]
 [ 0.2222981   0.20448652  0.06839564 ... -0.05685047  0.06249803
   0.01774155]
 [-0.2926893   0.05588254  0.08324507 ...  0.05507554  0.03377704
   0.00564649]]


In [41]:
nns

array([[79172, 15735, 22337, ..., 71849, 69015, 92811],
       [    0,     0,     0, ...,     0,     0,     0],
       [    0,     0,     0, ...,     0,     0,     0],
       ...,
       [    0,     0,     0, ...,     0,     0,     0],
       [    0,     0,     0, ...,     0,     0,     0],
       [    0,     0,     0, ...,     0,     0,     0]], dtype=uint32)

In [29]:
nns.shape

(10000, 10)

In [27]:
recall

0.00426

In [15]:
predict_categories_l2

,L2
1,7
2,1
3,1
4,8
5,7
...,...
99996,7
99997,0
99998,6
99999,7


In [85]:
li.model.predict(data_X_to_torch(data.iloc[1:2]))

array([3])

In [86]:
li.models[3].predict(data_X_to_torch(data.iloc[1:2]))

array([1])

In [93]:
data_navigation['category'] = pred_categories
data_navigation['category_L2'] = predict_categories_l2

In [94]:
data_navigation

,0,1,2,3,4,5,6,7,8,9,...,24,25,26,27,28,29,30,31,category,category_L2
1,0.142266,0.121459,-0.054467,-0.058987,0.049365,-0.107044,0.123232,0.003732,0.044797,-0.118452,...,-0.089511,0.077113,-0.118540,0.058264,0.065489,0.028578,-0.089683,0.126804,8,7
2,0.213798,-0.011502,-0.015170,0.149934,0.106344,0.082373,-0.120631,-0.061052,0.068679,0.033809,...,0.004503,-0.046667,-0.043632,-0.086331,-0.011793,-0.047739,0.020949,-0.030398,3,1
3,0.069708,-0.213904,0.132880,-0.150276,-0.049206,0.080040,-0.023638,0.017214,-0.208746,0.053342,...,-0.048399,-0.051265,0.019088,-0.036999,0.006381,0.011267,-0.029590,0.076341,7,1
4,0.136409,-0.124815,-0.154166,0.239908,0.002291,0.145442,-0.069386,-0.006642,0.124328,0.054876,...,0.072961,-0.035555,-0.003668,-0.007776,-0.065730,-0.013595,-0.012795,-0.030780,7,8
5,-0.389241,0.076076,-0.072001,0.103547,-0.058551,0.033468,0.017906,0.094761,0.034065,-0.220189,...,0.018730,0.050222,-0.021090,0.004709,0.043465,-0.049461,-0.052292,0.027773,9,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99996,0.126578,0.028599,-0.042934,0.001947,-0.202029,-0.128166,0.154031,0.023840,-0.009978,-0.046443,...,-0.004755,-0.051334,-0.058041,-0.034305,-0.015373,0.018450,-0.048168,0.030826,8,7
99997,-0.225225,-0.004567,-0.004400,-0.034974,-0.129672,0.113504,-0.003353,-0.057222,0.090702,-0.004591,...,-0.041191,0.046411,0.003835,0.022965,0.112995,0.092276,0.041511,-0.017332,6,0
99998,0.101137,0.037512,-0.051941,0.231012,0.011707,0.103784,0.142680,-0.054836,-0.078768,-0.032020,...,0.009569,-0.062856,0.009437,-0.065775,0.065454,0.017647,-0.066954,-0.031620,8,6
99999,-0.271447,0.021036,-0.050439,0.088891,-0.086671,-0.157237,0.029204,-0.001032,-0.122273,-0.076587,...,0.068649,-0.014509,0.040324,-0.038653,-0.043279,0.056283,-0.011639,-0.088651,9,7


In [108]:
pred_proba_categories_l2

(array([[0.73221105, 0.09233347, 0.07397919, ..., 0.0064247 , 0.00307956,
         0.00235663],
        [0.4276901 , 0.29809052, 0.08396686, ..., 0.01425164, 0.01110006,
         0.00513953],
        [0.23595823, 0.21585794, 0.16259144, ..., 0.03531529, 0.03447411,
         0.0180939 ],
        ...,
        [0.22499156, 0.17301367, 0.15733428, ..., 0.04091265, 0.04023259,
         0.02760508],
        [0.35422376, 0.19174714, 0.121807  , ..., 0.03566937, 0.0188215 ,
         0.01593618],
        [0.27764392, 0.20481573, 0.12898836, ..., 0.03525507, 0.02643807,
         0.02320394]], dtype=float32),
 array([[2, 3, 1, ..., 9, 0, 5],
        [3, 5, 9, ..., 0, 1, 4],
        [5, 9, 3, ..., 0, 1, 4],
        ...,
        [5, 3, 7, ..., 4, 0, 1],
        [5, 9, 7, ..., 2, 1, 4],
        [5, 7, 9, ..., 0, 1, 4]]))

In [112]:
cat[1]

1

In [ ]:
data_navigation

In [89]:
_, pred_proba_categories = li.model.predict_proba(
    data_X_to_torch(queries_navigation)
)

In [91]:
pred_proba_categories[:, 0]

array([6, 5, 0, ..., 8, 6, 6])

In [102]:
pred_proba_categories

array([[6, 4, 5, ..., 0, 3, 1],
       [5, 4, 6, ..., 7, 8, 3],
       [0, 9, 5, ..., 1, 6, 8],
       ...,
       [8, 9, 6, ..., 1, 0, 3],
       [6, 2, 4, ..., 3, 1, 8],
       [6, 5, 7, ..., 9, 0, 8]])

In [103]:
cat_idxs = np.where(pred_proba_categories[:, 0] == cat[0])[0]

In [ ]:
cat_idxs = np.where(pred_proba_categories[:, 0] == cat[0])[0]

In [ ]:
for cat, g in tqdm(data_navigation.groupby('category')):
    cat_idxs = np.where(pred_categories == cat)[0]

In [75]:
n_categories = 10
for cat in range(n_categories):
    cat_data = data.loc[pred_categories == cat]
    predict_categories_l2.loc[cat_data.index, 'L2'] = li.models[cat].predict(data_X_to_torch(cat_data))

NameError: name 'labels' is not defined

In [64]:
data.iloc[:1]

,0,1,2,3,4,5,6,7,8,9,...,22,23,24,25,26,27,28,29,30,31
1,0.142266,0.121459,-0.054467,-0.058987,0.049365,-0.107044,0.123232,0.003732,0.044797,-0.118452,...,0.063983,0.037666,-0.089511,0.077113,-0.11854,0.058264,0.065489,0.028578,-0.089683,0.126804


In [65]:
cat_data = data.loc[pred_categories == 1]

In [67]:
predict_categories_l2.loc[cat_data.index, 'L2'] = li.models[1].predict(data_X_to_torch(cat_data))


In [68]:
predict_categories_l2

,L2
1,9
2,3
3,4
4,5
5,9
...,...
99996,6
99997,0
99998,6
99999,0


### 100 vs. 200 categories

In [12]:
li = LearnedIndex()
pred_categories, build_t = li.build(
    data,
    n_categories=200,
    epochs=200,
    lr=0.1
)

[2023-07-14 15:16:26,976][INFO ][__main__.LearnedInde] Epochs: 200, step: 20
[2023-07-14 15:16:50,503][INFO ][__main__.LearnedInde] Epoch 20 | Loss 1.98601
[2023-07-14 15:17:12,825][INFO ][__main__.LearnedInde] Epoch 40 | Loss 1.18380
[2023-07-14 15:17:35,042][INFO ][__main__.LearnedInde] Epoch 60 | Loss 1.05507
[2023-07-14 15:17:57,317][INFO ][__main__.LearnedInde] Epoch 80 | Loss 1.10324
[2023-07-14 15:18:19,542][INFO ][__main__.LearnedInde] Epoch 100 | Loss 1.13756
[2023-07-14 15:18:41,839][INFO ][__main__.LearnedInde] Epoch 120 | Loss 0.89168
[2023-07-14 15:19:04,248][INFO ][__main__.LearnedInde] Epoch 140 | Loss 1.07258
[2023-07-14 15:19:26,428][INFO ][__main__.LearnedInde] Epoch 160 | Loss 0.81745
[2023-07-14 15:19:48,658][INFO ][__main__.LearnedInde] Epoch 180 | Loss 0.70368


In [13]:
LOG.info(f'Loading GT')
gt_path = f'data/groundtruth-{size}.h5'
f3 = h5py.File(gt_path, 'r')
loaded_gt = f3['knns'][:, :]

[2023-07-14 15:20:10,501][INFO ][__main__] Loading GT


In [14]:
k=10
bucket=8

In [15]:
dists, nns, search_t, inference_t, search_single_t, seq_search_t, pure_seq_search_t, sort_t = li.search(
    data_navigation=data,
    queries_navigation=queries,
    data_search=data_search,
    queries_search=queries_search,
    pred_categories=pred_categories,
    n_buckets=bucket,
    k=k,
    use_threshold=True
)
LOG.info('Inference time: %s', inference_t)
LOG.info('Search time: %s', search_t)
LOG.info('Search single time: %s', search_single_t)
LOG.info('Sequential search time: %s', seq_search_t)
LOG.info('Pure sequential search time: %s', pure_seq_search_t)
LOG.info('Sort time: %s', sort_t)

100%|██████████| 200/200 [00:01<00:00, 166.66it/s]
[2023-07-14 15:20:20,786][INFO ][__main__.LearnedInde] t_comp_threshold: 0.003280162811279297
[2023-07-14 15:20:20,789][INFO ][__main__] Inference time: 0.1276540756225586
[2023-07-14 15:20:20,792][INFO ][__main__] Search time: 10.176419019699097
[2023-07-14 15:20:20,795][INFO ][__main__] Search single time: 10.020214319229126
[2023-07-14 15:20:20,797][INFO ][__main__] Sequential search time: 8.815887451171875
[2023-07-14 15:20:20,800][INFO ][__main__] Pure sequential search time: 7.360041856765747
[2023-07-14 15:20:20,802][INFO ][__main__] Sort time: 0.37026500701904297


In [16]:
def get_recall(I, gt, k):
    assert k <= I.shape[1]
    assert len(I) == len(gt)

    n = len(I)
    recall = 0
    for i in range(n):
        recall += len(set(I[i, :k]) & set(gt[i, :k]))
    return recall / (n * k)

recall = get_recall(nns, loaded_gt, 10)


In [17]:
recall = get_recall(nns, loaded_gt, 10)


In [18]:
recall

0.7824

In [13]:
li = LearnedIndex()
pred_categories, build_t = li.build(
    data,
    n_categories=100,
    epochs=200,
    lr=0.1
)

[2023-07-14 15:11:44,421][INFO ][__main__.LearnedInde] Epochs: 200, step: 20
[2023-07-14 15:12:07,214][INFO ][__main__.LearnedInde] Epoch 20 | Loss 1.23865
[2023-07-14 15:12:28,505][INFO ][__main__.LearnedInde] Epoch 40 | Loss 0.85032
[2023-07-14 15:12:49,770][INFO ][__main__.LearnedInde] Epoch 60 | Loss 0.80628
[2023-07-14 15:13:10,952][INFO ][__main__.LearnedInde] Epoch 80 | Loss 0.67493
[2023-07-14 15:13:32,132][INFO ][__main__.LearnedInde] Epoch 100 | Loss 0.76835
[2023-07-14 15:13:53,377][INFO ][__main__.LearnedInde] Epoch 120 | Loss 0.83141
[2023-07-14 15:14:14,714][INFO ][__main__.LearnedInde] Epoch 140 | Loss 0.64362
[2023-07-14 15:14:36,058][INFO ][__main__.LearnedInde] Epoch 160 | Loss 0.73345
[2023-07-14 15:14:57,253][INFO ][__main__.LearnedInde] Epoch 180 | Loss 0.75338


In [17]:
k=10
bucket=4

In [18]:
dists, nns, search_t, inference_t, search_single_t, seq_search_t, pure_seq_search_t, sort_t = li.search(
    data_navigation=data,
    queries_navigation=queries,
    data_search=data_search,
    queries_search=queries_search,
    pred_categories=pred_categories,
    n_buckets=bucket,
    k=k,
    use_threshold=True
)
LOG.info('Inference time: %s', inference_t)
LOG.info('Search time: %s', search_t)
LOG.info('Search single time: %s', search_single_t)
LOG.info('Sequential search time: %s', seq_search_t)
LOG.info('Pure sequential search time: %s', pure_seq_search_t)
LOG.info('Sort time: %s', sort_t)

100%|██████████| 100/100 [00:01<00:00, 63.47it/s]
[2023-07-14 15:15:56,355][INFO ][__main__.LearnedInde] t_comp_threshold: 0.001416921615600586
[2023-07-14 15:15:56,358][INFO ][__main__] Inference time: 0.07615113258361816
[2023-07-14 15:15:56,360][INFO ][__main__] Search time: 6.98410439491272
[2023-07-14 15:15:56,363][INFO ][__main__] Search single time: 6.893368244171143
[2023-07-14 15:15:56,365][INFO ][__main__] Sequential search time: 5.824580192565918
[2023-07-14 15:15:56,368][INFO ][__main__] Pure sequential search time: 4.993402004241943
[2023-07-14 15:15:56,370][INFO ][__main__] Sort time: 0.6700947284698486


In [20]:
recall = get_recall(nns, loaded_gt, 10)
recall

0.7876